In [1]:
# !gdown 1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz
!gdown 1uBvcov6itBsoW5fX-hiF_EZrD7QeOi8z

Downloading...
From: https://drive.google.com/uc?id=1uBvcov6itBsoW5fX-hiF_EZrD7QeOi8z
To: /content/imputed_type_bangkok_traffy.csv
100% 358M/358M [00:01<00:00, 180MB/s]


In [2]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [3]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

mv: cannot move 'spark-3.3.2-bin-hadoop3' to 'spark/spark-3.3.2-bin-hadoop3': Directory not empty


In [4]:
import findspark
findspark.init()

In [5]:
spark_url = 'local'

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('time-taken-teller')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [8]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('time-taken-teller')\
        .getOrCreate()

In [9]:
path = 'imputed_type_bangkok_traffy.csv'
df = spark.read.csv(path, header=True, inferSchema=True,multiLine = True,escape="\"",sep=",",encoding='utf-8')


In [10]:
df.filter(col('type').isNull()).count()

97

In [11]:
df = df.dropna(subset=['type'])

In [12]:
df

DataFrame[_c0: int, ticket_id: string, type: string, organization: string, comment: string, photo: string, photo_after: string, coords: string, address: string, subdistrict: string, district: string, province: string, timestamp: timestamp, state: string, star: double, count_reopen: double, last_activity: timestamp]

In [13]:
# columns_to_drop = ['photo', 'photo_after', 'address', 'star']
columns_to_drop = ['ticket_id', 'organization','comment','photo','photo_after','coords','address','star','count_reopen']
df = df.drop(*columns_to_drop)

In [14]:
df.show(5)

+---+-------------------+-----------+--------+--------------------+--------------------+--------------+--------------------+
|_c0|               type|subdistrict|district|            province|           timestamp|         state|       last_activity|
+---+-------------------+-----------+--------+--------------------+--------------------+--------------+--------------------+
|  0|         {แสงสว่าง}|    บางพลัด| บางพลัด|       กรุงเทพมหานคร|2021-09-01 10:44:...|กำลังดำเนินการ|2022-02-22 04:59:...|
|  1|        {ความสะอาด}|       null|    null|       กรุงเทพมหานคร|2021-09-03 12:51:...|     เสร็จสิ้น|2022-06-04 15:34:...|
|  2|            {สายไฟ}|  สามเสนนอก|ห้วยขวาง|จังหวัดกรุงเทพมหานคร|2021-09-19 06:47:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  3|        {ถนน,สะพาน}|       สีลม|  บางรัก|       กรุงเทพมหานคร|2021-09-19 07:40:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|       กรุงเทพมหานคร|2021-09-19 14:56:...|     เสร็จสิ้น|2022-06-21 08:21:...|


In [15]:
from pyspark.sql.functions import avg, min, max, countDistinct, split, explode
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import col, udf

In [16]:
df.show(5), df.count()

+---+-------------------+-----------+--------+--------------------+--------------------+--------------+--------------------+
|_c0|               type|subdistrict|district|            province|           timestamp|         state|       last_activity|
+---+-------------------+-----------+--------+--------------------+--------------------+--------------+--------------------+
|  0|         {แสงสว่าง}|    บางพลัด| บางพลัด|       กรุงเทพมหานคร|2021-09-01 10:44:...|กำลังดำเนินการ|2022-02-22 04:59:...|
|  1|        {ความสะอาด}|       null|    null|       กรุงเทพมหานคร|2021-09-03 12:51:...|     เสร็จสิ้น|2022-06-04 15:34:...|
|  2|            {สายไฟ}|  สามเสนนอก|ห้วยขวาง|จังหวัดกรุงเทพมหานคร|2021-09-19 06:47:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  3|        {ถนน,สะพาน}|       สีลม|  บางรัก|       กรุงเทพมหานคร|2021-09-19 07:40:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|       กรุงเทพมหานคร|2021-09-19 14:56:...|     เสร็จสิ้น|2022-06-21 08:21:...|


(None, 269828)

In [17]:
df = df.filter((col('province').rlike('กรุงเทพ|bangkok')) & (col('province').isNotNull()))
df = df.drop(*['province'])

In [18]:
# df.show(5), df.count()

In [19]:
df = df.dropna(subset=['subdistrict', 'district'])
df.show(5), df.count()

+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
|_c0|                type|subdistrict| district|           timestamp|         state|       last_activity|
+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
|  0|          {แสงสว่าง}|    บางพลัด|  บางพลัด|2021-09-01 10:44:...|กำลังดำเนินการ|2022-02-22 04:59:...|
|  2|             {สายไฟ}|  สามเสนนอก| ห้วยขวาง|2021-09-19 06:47:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  3|         {ถนน,สะพาน}|       สีลม|   บางรัก|2021-09-19 07:40:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  4| {น้ำท่วม,ร้องเรียน}|    หนองบอน|   ประเวศ|2021-09-19 14:56:...|     เสร็จสิ้น|2022-06-21 08:21:...|
|  5|{ถนน,ร้องเรียน,น้...|     ขุมทอง|ลาดกระบัง|2021-09-20 00:24:...|กำลังดำเนินการ|2022-02-22 04:29:...|
+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
only showing top 5 rows



(None, 268795)

In [21]:
df.show(5), df.count()

+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
|_c0|                type|subdistrict| district|           timestamp|         state|       last_activity|
+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
|  0|          {แสงสว่าง}|    บางพลัด|  บางพลัด|2021-09-01 10:44:...|กำลังดำเนินการ|2022-02-22 04:59:...|
|  2|             {สายไฟ}|  สามเสนนอก| ห้วยขวาง|2021-09-19 06:47:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  3|         {ถนน,สะพาน}|       สีลม|   บางรัก|2021-09-19 07:40:...|กำลังดำเนินการ|2022-02-22 04:30:...|
|  4| {น้ำท่วม,ร้องเรียน}|    หนองบอน|   ประเวศ|2021-09-19 14:56:...|     เสร็จสิ้น|2022-06-21 08:21:...|
|  5|{ถนน,ร้องเรียน,น้...|     ขุมทอง|ลาดกระบัง|2021-09-20 00:24:...|กำลังดำเนินการ|2022-02-22 04:29:...|
+---+--------------------+-----------+---------+--------------------+--------------+--------------------+
only showing top 5 rows



(None, 268795)

In [22]:
df = df.filter(col('state').rlike('เสร็จ'))

In [23]:
from pyspark.sql.functions import col, datediff

# Convert 'timestamp' and 'last_activity' columns to datetime
df = df.withColumn('timestamp', col('timestamp').cast('timestamp'))
df = df.withColumn('last_activity', col('last_activity').cast('timestamp'))

# Calculate the difference in days
df = df.withColumn('time', datediff(col('last_activity'), col('timestamp')))

In [24]:
df.show(5)

+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+
|_c0|               type|subdistrict|district|           timestamp|    state|       last_activity|time|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|2021-09-19 14:56:...|เสร็จสิ้น|2022-06-21 08:21:...| 275|
| 10|            {สะพาน}|    ยานนาวา|    สาทร|2021-09-26 05:03:...|เสร็จสิ้น|2022-06-06 01:17:...| 253|
| 25|      {น้ำท่วม,ถนน}|   ลาดพร้าว|ลาดพร้าว|2021-12-09 12:29:...|เสร็จสิ้น|2022-08-12 07:18:...| 246|
| 64|            {สายไฟ}|   ลาดพร้าว|ลาดพร้าว|2021-12-13 05:53:...|เสร็จสิ้น|2023-03-14 12:09:...| 456|
|102|       {เสียงรบกวน}|      ดุสิต|   ดุสิต|2021-12-17 08:46:...|เสร็จสิ้น|2022-06-01 03:16:...| 166|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+
only showing top 5 rows



In [25]:
from pyspark.sql.functions import col, month, year, quarter, dayofweek

# Extract the month
df = df.withColumn('month', month(col('timestamp')))

# Extract the year
df = df.withColumn('year', year(col('timestamp')))

# Extract the quarter of the year
df = df.withColumn('quarter', quarter(col('timestamp')))

# Determine if the day is a weekend
df = df.withColumn('is_weekend', (dayofweek(col('timestamp')) >= 6).cast('integer'))

In [26]:
df.show(5)

+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|_c0|               type|subdistrict|district|           timestamp|    state|       last_activity|time|month|year|quarter|is_weekend|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|2021-09-19 14:56:...|เสร็จสิ้น|2022-06-21 08:21:...| 275|    9|2021|      3|         0|
| 10|            {สะพาน}|    ยานนาวา|    สาทร|2021-09-26 05:03:...|เสร็จสิ้น|2022-06-06 01:17:...| 253|    9|2021|      3|         0|
| 25|      {น้ำท่วม,ถนน}|   ลาดพร้าว|ลาดพร้าว|2021-12-09 12:29:...|เสร็จสิ้น|2022-08-12 07:18:...| 246|   12|2021|      4|         0|
| 64|            {สายไฟ}|   ลาดพร้าว|ลาดพร้าว|2021-12-13 05:53:...|เสร็จสิ้น|2023-03-14 12:09:...| 456|   12|2021|      4|         0|
|102|       {เสียงรบกวน}|      ดุสิต|   ดุสิต|2021-12-17 08:46

In [27]:
df.dtypes

[('_c0', 'int'),
 ('type', 'string'),
 ('subdistrict', 'string'),
 ('district', 'string'),
 ('timestamp', 'timestamp'),
 ('state', 'string'),
 ('last_activity', 'timestamp'),
 ('time', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('quarter', 'int'),
 ('is_weekend', 'int')]

In [30]:
df.show(5)

+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|_c0|               type|subdistrict|district|           timestamp|    state|       last_activity|time|month|year|quarter|is_weekend|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|2021-09-19 14:56:...|เสร็จสิ้น|2022-06-21 08:21:...| 275|    9|2021|      3|         0|
| 10|            {สะพาน}|    ยานนาวา|    สาทร|2021-09-26 05:03:...|เสร็จสิ้น|2022-06-06 01:17:...| 253|    9|2021|      3|         0|
| 25|      {น้ำท่วม,ถนน}|   ลาดพร้าว|ลาดพร้าว|2021-12-09 12:29:...|เสร็จสิ้น|2022-08-12 07:18:...| 246|   12|2021|      4|         0|
| 64|            {สายไฟ}|   ลาดพร้าว|ลาดพร้าว|2021-12-13 05:53:...|เสร็จสิ้น|2023-03-14 12:09:...| 456|   12|2021|      4|         0|
|102|       {เสียงรบกวน}|      ดุสิต|   ดุสิต|2021-12-17 08:46

In [31]:
df.columns

['_c0',
 'type',
 'subdistrict',
 'district',
 'timestamp',
 'state',
 'last_activity',
 'time',
 'month',
 'year',
 'quarter',
 'is_weekend']

In [32]:
df.na.drop(subset=['type',
 'subdistrict',
 'district',
 'timestamp',
 'state',
 'last_activity',
 'time',
 'month',
 'year',
 'quarter',
 'is_weekend'])

DataFrame[_c0: int, type: string, subdistrict: string, district: string, timestamp: timestamp, state: string, last_activity: timestamp, time: int, month: int, year: int, quarter: int, is_weekend: int]

In [33]:
df.show(5)

+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|_c0|               type|subdistrict|district|           timestamp|    state|       last_activity|time|month|year|quarter|is_weekend|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|2021-09-19 14:56:...|เสร็จสิ้น|2022-06-21 08:21:...| 275|    9|2021|      3|         0|
| 10|            {สะพาน}|    ยานนาวา|    สาทร|2021-09-26 05:03:...|เสร็จสิ้น|2022-06-06 01:17:...| 253|    9|2021|      3|         0|
| 25|      {น้ำท่วม,ถนน}|   ลาดพร้าว|ลาดพร้าว|2021-12-09 12:29:...|เสร็จสิ้น|2022-08-12 07:18:...| 246|   12|2021|      4|         0|
| 64|            {สายไฟ}|   ลาดพร้าว|ลาดพร้าว|2021-12-13 05:53:...|เสร็จสิ้น|2023-03-14 12:09:...| 456|   12|2021|      4|         0|
|102|       {เสียงรบกวน}|      ดุสิต|   ดุสิต|2021-12-17 08:46

In [34]:
column_names = df.columns
print(column_names)

['_c0', 'type', 'subdistrict', 'district', 'timestamp', 'state', 'last_activity', 'time', 'month', 'year', 'quarter', 'is_weekend']


In [35]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- subdistrict: string (nullable = true)
 |-- district: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- state: string (nullable = true)
 |-- last_activity: timestamp (nullable = true)
 |-- time: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- quarter: integer (nullable = true)
 |-- is_weekend: integer (nullable = true)



In [36]:
df.show(10)

+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|_c0|               type|subdistrict|district|           timestamp|    state|       last_activity|time|month|year|quarter|is_weekend|
+---+-------------------+-----------+--------+--------------------+---------+--------------------+----+-----+----+-------+----------+
|  4|{น้ำท่วม,ร้องเรียน}|    หนองบอน|  ประเวศ|2021-09-19 14:56:...|เสร็จสิ้น|2022-06-21 08:21:...| 275|    9|2021|      3|         0|
| 10|            {สะพาน}|    ยานนาวา|    สาทร|2021-09-26 05:03:...|เสร็จสิ้น|2022-06-06 01:17:...| 253|    9|2021|      3|         0|
| 25|      {น้ำท่วม,ถนน}|   ลาดพร้าว|ลาดพร้าว|2021-12-09 12:29:...|เสร็จสิ้น|2022-08-12 07:18:...| 246|   12|2021|      4|         0|
| 64|            {สายไฟ}|   ลาดพร้าว|ลาดพร้าว|2021-12-13 05:53:...|เสร็จสิ้น|2023-03-14 12:09:...| 456|   12|2021|      4|         0|
|102|       {เสียงรบกวน}|      ดุสิต|   ดุสิต|2021-12-17 08:46

In [37]:
df.dtypes

[('_c0', 'int'),
 ('type', 'string'),
 ('subdistrict', 'string'),
 ('district', 'string'),
 ('timestamp', 'timestamp'),
 ('state', 'string'),
 ('last_activity', 'timestamp'),
 ('time', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('quarter', 'int'),
 ('is_weekend', 'int')]

In [38]:
# def array_to_string(my_list):
#     return '[' + ','.join([str(elem) for elem in my_list]) + ']'

# array_to_string_udf = udf(array_to_string, StringType())

# df = df.withColumn('type', array_to_string_udf(df["type"]))

In [39]:
df.filter(col('type').isNull()).count()

0

In [40]:
df.dtypes

[('_c0', 'int'),
 ('type', 'string'),
 ('subdistrict', 'string'),
 ('district', 'string'),
 ('timestamp', 'timestamp'),
 ('state', 'string'),
 ('last_activity', 'timestamp'),
 ('time', 'int'),
 ('month', 'int'),
 ('year', 'int'),
 ('quarter', 'int'),
 ('is_weekend', 'int')]

In [50]:
columns_to_drop = ['_c0']
df = df.drop(*columns_to_drop)

In [52]:
# Convert the transformed Spark DataFrame to a Pandas DataFrame
pandasDF = df.toPandas()

# Print the Pandas DataFrame
print(pandasDF)

/content/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)
/content/spark/python/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


                          type subdistrict  district  \
0          {น้ำท่วม,ร้องเรียน}     หนองบอน    ประเวศ   
1                      {สะพาน}     ยานนาวา      สาทร   
2                {น้ำท่วม,ถนน}    ลาดพร้าว  ลาดพร้าว   
3                      {สายไฟ}    ลาดพร้าว  ลาดพร้าว   
4                 {เสียงรบกวน}       ดุสิต     ดุสิต   
...                        ...         ...       ...   
201626               {ทางเท้า}   บางยี่ขัน   บางพลัด   
201627                 {จราจร}   วงศ์สว่าง   บางซื่อ   
201628                 {จราจร}   วงศ์สว่าง   บางซื่อ   
201629  {ถนน,จราจร,สะพาน,คลอง}   วงศ์สว่าง   บางซื่อ   
201630    {ความสะอาด,ถนน,คลอง}    คลองจั่น   บางกะปิ   

                        timestamp      state              last_activity  time  \
0      2021-09-19 14:56:08.924992  เสร็จสิ้น 2022-06-21 08:21:09.532782   275   
1      2021-09-26 05:03:52.594898  เสร็จสิ้น 2022-06-06 01:17:12.272904   253   
2      2021-12-09 12:29:08.408763  เสร็จสิ้น 2022-08-12 07:18:44.884945   246   
3  